In [1]:
import astropy.units as u
from astroduet.duet_telescope import load_telescope_parameters
from astroduet.duet_sensitivity import src_rate, bgd_sky_qe_rate, bgd_electronics, calc_exposure
from astroduet.bbmag import bb_abmag_fluence
import numpy as np
from matplotlib import pyplot as plt
from astroduet.bbmag import bb_abmag_fluence, bb_abmag

import astroduet.config as config
from astroduet.background import background_pixel_rate


%load_ext autoreload
%autoreload 2

from astropy.visualization import quantity_support
import matplotlib
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

In [2]:

# Band1
# 5-sigma limiting magnitude in 1, 2, and 3 stacked frames.

# Account for the fact that you're co-adding the two frames here:
duet = config.Telescope()


#tot_bgd_rate = bgd_band1 + bgd_band2
bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 300*u.s
print()
siglimit=5
dmag = 0.05
print()
for filter_type in ['B']:
    print(filter_type)
    [bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True, filter_type=filter_type, airglow_level='COS')
    tot_bgd_rate = bgd_band1

    for nframes in [1, 2, 3]:
        snr = 100
        swiftmag = 20 
        while snr > siglimit:
            swiftmag += dmag
            band1_fluence, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, filter_type=filter_type)
            band1_rate = duet.fluence_to_rate(band1_fluence)
            band2_rate = duet.fluence_to_rate(band2_fluence)
            src_rate = band1_rate

            snr = duet.calc_snr(exposure, src_rate, tot_bgd_rate, nint=nframes)
        bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone, bandtwo=bandtwo)        
        print('Band1 {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag1))






B
Band1 300.0 s 5-σ magnitude limit: 21.780542480741897 mag(AB)
Band1 600.0 s 5-σ magnitude limit: 22.2305424807419 mag(AB)
Band1 900.0 s 5-σ magnitude limit: 22.480542480741907 mag(AB)


In [3]:

# Band2
# 5-sigma limiting magnitude in 1, 2, and 3 stacked frames.

duet = config.Telescope()


#tot_bgd_rate = bgd_band1 + bgd_band2
bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 300*u.s
print()
siglimit=5
dmag = 0.05
print()
for filter_type in ['B']:
    print(filter_type)
    [bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True, filter_type=filter_type, airglow_level='COS')
    tot_bgd_rate = bgd_band2

    for nframes in [1, 2, 3]:
        snr = 100
        swiftmag = 20 
        while snr > siglimit:
            swiftmag += dmag
            band1_fluence, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, filter_type=filter_type)
            band1_rate = duet.fluence_to_rate(band1_fluence)
            band2_rate = duet.fluence_to_rate(band2_fluence)
            src_rate = band2_rate

            snr = duet.calc_snr(exposure, src_rate, tot_bgd_rate, nint=nframes)
        bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone, bandtwo=bandtwo)        
        print('Band2 {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag2))







B
Band2 300.0 s 5-σ magnitude limit: 21.141224059720265 mag(AB)
Band2 600.0 s 5-σ magnitude limit: 21.59122405972027 mag(AB)
Band2 900.0 s 5-σ magnitude limit: 21.79122405972027 mag(AB)


In [4]:

# Stacked channels
# 5-sigma limiting magnitude in 1, 2, 3, and 4 stacked frames.

# Account for the fact that you're co-adding the two frames here:
duet = config.Telescope()
duet.read_noise = (2**0.5) * duet.read_noise


#tot_bgd_rate = bgd_band1 + bgd_band2
bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 300*u.s
print()
siglimit=5
dmag = 0.05
print()
for filter_type in ['B']:
    print(filter_type)
    [bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True, filter_type=filter_type)
    tot_bgd_rate = bgd_band1 + bgd_band2

    for nframes in [1, 2, 3]:
        snr = 100
        swiftmag = 20 
        while snr > siglimit:
            swiftmag += dmag
            band1_fluence, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, filter_type=filter_type)
            band1_rate = duet.fluence_to_rate(band1_fluence)
            band2_rate = duet.fluence_to_rate(band2_fluence)
            src_rate = band1_rate + band2_rate

            snr = duet.calc_snr(exposure, src_rate, tot_bgd_rate, nint=nframes)
        bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone, bandtwo=bandtwo)        
        print('Stacked Bands {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag1))







B
Stacked Bands 300.0 s 5-σ magnitude limit: 21.930542480741895 mag(AB)
Stacked Bands 600.0 s 5-σ magnitude limit: 22.3305424807419 mag(AB)
Stacked Bands 900.0 s 5-σ magnitude limit: 22.580542480741908 mag(AB)


In [5]:
# FOM: Time to 22 ABmag for this configuration for DUET1
# Old FOM for this 
duet = config.Telescope()


bandone = duet.bandpass1
bandtwo = duet.bandpass2

siglimit=10
swiftmag = 20
dmag = 0.05
bbmag1 = 5*u.ABmag
bbmag_target = 22*u.ABmag
while bbmag1 < bbmag_target:
    swiftmag+=dmag
    bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone, bandtwo=bandtwo)


# Low Zodi, 10-sigma case
[bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True)
band1_fluence, foo = bb_abmag_fluence(duet = duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K)
band1_rate = duet.fluence_to_rate(band1_fluence)
req_exp = calc_exposure(siglimit, band1_rate, bgd_band1, duet.read_noise, duet.neff)
print('Low-Zodi {}-sig, {} seconds'.format(siglimit, req_exp.value))


# High Zodi, 10-sigma case.
siglimit=10
[bgd_band1, bgd_band2] = background_pixel_rate(duet, high_zodi=True)
band1_fluence, foo = bb_abmag_fluence(duet = duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K)
band1_rate = duet.fluence_to_rate(band1_fluence)
req_exp = calc_exposure(siglimit, band1_rate, bgd_band1, duet.read_noise, duet.neff)
print('High-Zodi {}-sig, {} seconds'.format(siglimit, req_exp.value))

# High Zodi, 5-sigma case
siglimit=5
[bgd_band1, bgd_band2] = background_pixel_rate(duet, high_zodi=True)
band1_fluence, foo = bb_abmag_fluence(duet = duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K)
band1_rate = duet.fluence_to_rate(band1_fluence)

req_exp = calc_exposure(siglimit, band1_rate, bgd_band1, duet.read_noise, duet.neff)
print('High-Zodi {}-sig, {} seconds'.format(siglimit, req_exp.value))




Low-Zodi 10-sig, 1152.6427137578592 seconds
High-Zodi 10-sig, 1736.3936879277696 seconds
High-Zodi 5-sig, 539.4962128620656 seconds
